# 从零实现逻辑回归

## 学习目标

通过本notebook，你将学会：
- 理解逻辑回归的基本原理和Sigmoid函数
- 从零实现逻辑回归算法（包括梯度下降优化）
- 理解交叉熵损失函数和最大似然估计
- 掌握模型评估方法（准确率、ROC曲线）
- 可视化决策边界和训练过程

## 课程概述

本notebook将带你从零开始实现逻辑回归算法，包括：
1. **Sigmoid函数**：将线性输出转换为概率
2. **交叉熵损失函数**：用于二分类问题的损失函数
3. **梯度下降优化**：迭代优化参数
4. **正则化**：L1和L2正则化防止过拟合

我们将通过可视化和实验来理解逻辑回归的工作原理。


## 1. 环境准备

### 版本要求
- Python >= 3.7
- NumPy >= 1.19.0
- Matplotlib >= 3.3.0
- scikit-learn >= 0.24.0

### 导入库


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc

# 设置中文字体（如果需要显示中文）
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子，确保结果可复现
np.random.seed(42)

# 设置matplotlib在notebook中内联显示
%matplotlib inline

print("环境准备完成！")
print(f"NumPy版本: {np.__version__}")


## 2. 理论回顾

### 逻辑回归的基本原理

逻辑回归是用于**二分类**问题的线性模型，它通过Sigmoid函数将线性输出转换为概率。

**数学形式**：
$$h_\theta(x) = \sigma(\theta^T x) = \frac{1}{1 + e^{-\theta^T x}}$$

其中：
- $\sigma(z)$ 是Sigmoid函数
- $\theta$ 是模型参数（权重和偏置）
- $x$ 是输入特征

### 损失函数：交叉熵

逻辑回归使用**交叉熵损失函数**（Cross-Entropy Loss）：

$$J(\theta) = -\frac{1}{m}\sum_{i=1}^{m} [y^{(i)} \log h_\theta(x^{(i)}) + (1-y^{(i)}) \log(1 - h_\theta(x^{(i)}))]$$

这个损失函数来自**最大似然估计**（MLE），目标是最大化数据的似然概率。

### 梯度下降

梯度更新公式：
$$\theta_j := \theta_j - \alpha \frac{\partial J}{\partial \theta_j}$$

其中：
$$\frac{\partial J}{\partial \theta_j} = \frac{1}{m}\sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)}$$


## 3. 实现LogisticRegression类

现在让我们从零实现逻辑回归算法。


In [ ]:
class LogisticRegression:
    """逻辑回归类（从零实现）"""
    
    def __init__(self, learning_rate=0.01, max_iter=1000, tol=1e-6, 
                 regularization=None, lambda_reg=0.1):
        """
        初始化逻辑回归模型
        
        参数:
        - learning_rate: 学习率（默认0.01）
        - max_iter: 最大迭代次数（默认1000）
        - tol: 收敛容差（默认1e-6）
        - regularization: 正则化类型，'l1'、'l2'或None（默认None）
        - lambda_reg: 正则化系数（默认0.1）
        """
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.regularization = regularization
        self.lambda_reg = lambda_reg
        self.theta = None  # 模型参数（权重和偏置）
        self.cost_history = []  # 记录每次迭代的损失值
    
    def sigmoid(self, z):
        """
        Sigmoid函数：将任意实数映射到(0, 1)区间
        
        参数:
        - z: 输入值（可以是标量、向量或矩阵）
        
        返回:
        - Sigmoid函数值
        """
        # 防止数值溢出：将z限制在合理范围内
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    def compute_cost(self, X, y):
        """
        计算损失函数（交叉熵）
        
        参数:
        - X: 特征矩阵，形状为(m, n+1)，已包含偏置项
        - y: 标签向量，形状为(m,)
        
        返回:
        - 损失值（标量）
        """
        m = X.shape[0]  # 样本数量
        h = self.sigmoid(X @ self.theta)  # 预测概率
        
        # 交叉熵损失
        # 添加小常数1e-15避免log(0)
        cost = -(1/m) * np.sum(y * np.log(h + 1e-15) + (1-y) * np.log(1-h + 1e-15))
        
        # 正则化项
        if self.regularization == 'l2':
            # L2正则化：惩罚参数平方和
            cost += (self.lambda_reg / (2*m)) * np.sum(self.theta[1:]**2)
        elif self.regularization == 'l1':
            # L1正则化：惩罚参数绝对值之和
            cost += (self.lambda_reg / m) * np.sum(np.abs(self.theta[1:]))
        
        return cost
    
    def compute_gradient(self, X, y):
        """
        计算梯度
        
        参数:
        - X: 特征矩阵，形状为(m, n+1)
        - y: 标签向量，形状为(m,)
        
        返回:
        - 梯度向量，形状为(n+1,)
        """
        m = X.shape[0]
        h = self.sigmoid(X @ self.theta)  # 预测概率
        
        # 基本梯度：交叉熵损失对参数的导数
        gradient = (1/m) * X.T @ (h - y)
        
        # 正则化项的梯度
        if self.regularization == 'l2':
            # L2正则化的梯度：lambda * theta
            gradient[1:] += (self.lambda_reg / m) * self.theta[1:]
        elif self.regularization == 'l1':
            # L1正则化的次梯度：lambda * sign(theta)
            gradient[1:] += (self.lambda_reg / m) * np.sign(self.theta[1:])
        
        return gradient
    
    def fit(self, X, y):
        """
        训练模型：使用梯度下降优化参数
        
        参数:
        - X: 特征矩阵，形状为(m, n)
        - y: 标签向量，形状为(m,)
        
        返回:
        - self: 返回自身，支持链式调用
        """
        # 添加偏置项：在特征矩阵前加一列1
        m, n = X.shape
        X_with_bias = np.hstack([np.ones((m, 1)), X])
        
        # 初始化参数：全零初始化
        self.theta = np.zeros(n + 1)
        
        # 梯度下降迭代
        for i in range(self.max_iter):
            # 计算当前损失
            cost = self.compute_cost(X_with_bias, y)
            self.cost_history.append(cost)
            
            # 计算梯度
            gradient = self.compute_gradient(X_with_bias, y)
            
            # 更新参数：沿着梯度反方向更新
            self.theta -= self.learning_rate * gradient
            
            # 检查收敛：如果损失变化小于容差，提前停止
            if i > 0 and abs(self.cost_history[-2] - self.cost_history[-1]) < self.tol:
                print(f"在第 {i+1} 次迭代后收敛")
                break
        
        return self
    
    def predict_proba(self, X):
        """
        预测概率：返回每个样本属于正类的概率
        
        参数:
        - X: 特征矩阵，形状为(m, n)
        
        返回:
        - 概率向量，形状为(m,)，每个值在[0, 1]之间
        """
        m = X.shape[0]
        X_with_bias = np.hstack([np.ones((m, 1)), X])
        return self.sigmoid(X_with_bias @ self.theta)
    
    def predict(self, X, threshold=0.5):
        """
        预测类别：根据概率和阈值预测类别
        
        参数:
        - X: 特征矩阵，形状为(m, n)
        - threshold: 决策阈值（默认0.5）
        
        返回:
        - 预测标签，形状为(m,)，值为0或1
        """
        probabilities = self.predict_proba(X)
        return (probabilities >= threshold).astype(int)
    
    def plot_cost_history(self):
        """绘制损失函数收敛曲线"""
        plt.figure(figsize=(10, 6))
        plt.plot(self.cost_history)
        plt.xlabel('迭代次数')
        plt.ylabel('损失值')
        plt.title('损失函数收敛曲线')
        plt.grid(True, alpha=0.3)
        plt.show()

print("LogisticRegression类定义完成！")


In [ ]:
# 生成二分类数据
# n_samples: 样本数量
# n_features: 特征数量
# n_redundant: 冗余特征数量（与信息特征线性相关）
# n_informative: 信息特征数量（对分类有用的特征）
# n_clusters_per_class: 每个类别的聚类数量
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1, 
                           random_state=42)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征维度: {X_train.shape[1]}")
print(f"\n类别分布:")
print(f"  类别0: {np.sum(y_train == 0)} 个样本")
print(f"  类别1: {np.sum(y_train == 1)} 个样本")


## 5. 模型训练

使用梯度下降训练逻辑回归模型。


In [ ]:
# 创建模型实例
# learning_rate: 学习率，控制参数更新步长
# max_iter: 最大迭代次数
# regularization: 正则化类型，'l2'表示L2正则化
# lambda_reg: 正则化系数，控制正则化强度
model = LogisticRegression(learning_rate=0.1, max_iter=1000, 
                          regularization='l2', lambda_reg=0.1)

# 训练模型
print("开始训练模型...")
model.fit(X_train, y_train)
print("模型训练完成！")

# 查看训练后的参数
print(f"\n模型参数:")
print(f"  偏置项 (theta[0]): {model.theta[0]:.4f}")
print(f"  权重 (theta[1:]): {model.theta[1:]}")
print(f"  最终损失: {model.cost_history[-1]:.4f}")


## 6. 模型评估

评估模型在测试集上的性能。


In [ ]:
# 预测测试集
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f"测试集准确率: {accuracy:.4f}")

# 计算ROC曲线和AUC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc:.4f}")

# 显示前10个样本的预测结果
print("\n前10个样本的预测结果:")
print("真实标签 | 预测标签 | 预测概率")
print("-" * 35)
for i in range(min(10, len(y_test))):
    print(f"   {y_test[i]}     |    {y_pred[i]}     |  {y_proba[i]:.4f}")


## 7. 可视化结果

可视化损失函数收敛曲线、决策边界和ROC曲线。


In [ ]:
# 创建图形
fig = plt.figure(figsize=(15, 5))

# 1. 损失函数收敛曲线
plt.subplot(1, 3, 1)
plt.plot(model.cost_history)
plt.xlabel('迭代次数')
plt.ylabel('损失值')
plt.title('损失函数收敛曲线')
plt.grid(True, alpha=0.3)

# 2. 决策边界
plt.subplot(1, 3, 2)
# 创建网格点用于绘制决策边界
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# 预测网格点的概率
Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# 绘制决策边界和概率等高线
plt.contourf(xx, yy, Z, levels=50, alpha=0.5, cmap='RdYlBu')
plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2, linestyles='--')
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='RdYlBu', 
           edgecolors='black', s=30, alpha=0.7)
plt.xlabel('特征1')
plt.ylabel('特征2')
plt.title('决策边界（训练集）')
plt.colorbar(label='预测概率')

# 3. ROC曲线
plt.subplot(1, 3, 3)
plt.plot(fpr, tpr, label=f'ROC曲线 (AUC = {roc_auc:.2f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='随机分类器')
plt.xlabel('假正例率 (FPR)')
plt.ylabel('真正例率 (TPR)')
plt.title('ROC曲线')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 8. 参数实验

尝试不同的学习率和正则化参数，观察对模型性能的影响。


In [ ]:
# 实验不同的学习率
learning_rates = [0.001, 0.01, 0.1, 1.0]
results = []

for lr in learning_rates:
    model_exp = LogisticRegression(learning_rate=lr, max_iter=1000, 
                                   regularization='l2', lambda_reg=0.1)
    model_exp.fit(X_train, y_train)
    y_pred_exp = model_exp.predict(X_test)
    accuracy_exp = accuracy_score(y_test, y_pred_exp)
    results.append({
        'learning_rate': lr,
        'accuracy': accuracy_exp,
        'final_cost': model_exp.cost_history[-1],
        'iterations': len(model_exp.cost_history)
    })
    print(f"学习率 {lr:6.3f}: 准确率={accuracy_exp:.4f}, "
          f"最终损失={model_exp.cost_history[-1]:.4f}, "
          f"迭代次数={len(model_exp.cost_history)}")

# 可视化结果
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 准确率对比
axes[0].bar(range(len(results)), [r['accuracy'] for r in results])
axes[0].set_xticks(range(len(results)))
axes[0].set_xticklabels([f"{r['learning_rate']}" for r in results])
axes[0].set_xlabel('学习率')
axes[0].set_ylabel('准确率')
axes[0].set_title('不同学习率对准确率的影响')
axes[0].grid(True, alpha=0.3)

# 最终损失对比
axes[1].bar(range(len(results)), [r['final_cost'] for r in results])
axes[1].set_xticks(range(len(results)))
axes[1].set_xticklabels([f"{r['learning_rate']}" for r in results])
axes[1].set_xlabel('学习率')
axes[1].set_ylabel('最终损失')
axes[1].set_title('不同学习率对最终损失的影响')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 9. 总结与思考

### 关键知识点总结

1. **Sigmoid函数**：将线性输出转换为概率，值域为(0, 1)
2. **交叉熵损失**：用于二分类问题的损失函数，来自最大似然估计
3. **梯度下降**：迭代优化参数，需要合适的学习率
4. **正则化**：L1和L2正则化可以防止过拟合

### 思考问题

1. **为什么逻辑回归使用交叉熵而不是均方误差？**
   - 提示：考虑概率解释和梯度性质

2. **学习率对训练有什么影响？**
   - 提示：观察不同学习率下的收敛曲线

3. **决策边界是如何形成的？**
   - 提示：思考Sigmoid函数和阈值的关系

4. **正则化如何影响模型？**
   - 提示：尝试不同的lambda_reg值，观察参数和性能变化

### 下一步学习

- 学习多分类逻辑回归（Softmax）
- 学习使用scikit-learn实现逻辑回归
- 学习处理类别不平衡问题
- 学习特征工程和特征选择
